In [139]:
#process images
from PIL import Image
import os
from os import listdir
from os.path import isdir, join
from datetime import datetime
begin = datetime.now()
root = r'\\fas3000-b.bei.is.keysight.com\v6\device_recognize\origin'
root_resized = r'C:\Zdata\DL pictures\Processed'

file_indexes = {}
for t in ['A', 'B']:
    date_indexes = {}
    for date in ['1016', '1017', '1018']:
        date_indexes[date] = 1
    file_indexes[t] = date_indexes

def process_image(image_file):
    im = Image.open(image_file)
    im = im.convert('L')    
    x, y = im.size
    if (y > x):
        im = im.rotate(90, expand=True)        
    x, y = im.size    
    resize_y = 168
    resize_x = 168 * x / y
    im.thumbnail((resize_x, resize_y), Image.ANTIALIAS)    
    target_x = 224
    target_y = 168
    new_size = (resize_x/2 - target_x/2, resize_y/2 - target_y/2, resize_x/2 + target_x/2, resize_y/2 + target_y/2)
    im = im.crop(new_size)    
    return im

# types = [f for f in listdir(root) if isdir(join(root, f))]
for t in ['A', 'B']:
    print(t)
    type_folder = join(root, t)
    users = [f for f in listdir(type_folder) if isdir(join(type_folder, f))]
    for user in users:
        print(user)
        user_folder = join(type_folder, user)
        onoffs = [f for f in listdir(user_folder) if isdir(join(user_folder, f))]
        for onoff in onoffs:
            onoff_folder = join(user_folder, onoff)
            with os.scandir(onoff_folder) as pictures:
                for picture in pictures:                      
                    stat = picture.stat()
                    date = datetime.fromtimestamp(stat.st_mtime)
                    if not picture.name.endswith('.jpg'):
                        continue
                    im = process_image(join(onoff_folder, picture.name))
                    dateString = "%2d%2d"%(date.month, date.day)
                    index = file_indexes[t][dateString]
                    im.save(join(root_resized, "%s_%s_%04d.jpg"%(t, dateString, index)), "JPEG")
                    file_indexes[t][dateString] = index + 1
                    
elapse = datetime.now() - begin
print(elapse)

A
mingfang
yukunh01
jianji01
yiliang1
pengzh01
xinchen
mingmyao
yueguo01
cn569306
xiongan2
yingying
B
mingfang
yukunh01
jianji01
yiliang1
pengzh01
xinchen
mingmyao
yueguo01
cn569306
yingying
xiongan2
0:33:37.657142


In [140]:
#split data set into training and test
import os
from os import listdir
from os.path import isdir, join
from datetime import datetime
import random
import shutil

begin = datetime.now()
processed_folder = r'C:\Zdata\DL pictures\Processed'
training_folder = join(processed_folder, 'training')
test_folder = join(processed_folder, 'test')

def create_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

def classify_image_files(category, date, user, indexes, destination_folder):
    create_folder(destination_folder)
    for index in indexes:
        file_name = "%s_%s_%04d.jpg"%(category, date, index)
        source_file = join(processed_folder, file_name)
        destination_file = join(destination_folder, file_name)
        shutil.move(source_file, destination_file)

for category in ['A', 'B']:
    for date in ['1016', '1017', '1018']:
        indexes = list(range(1, 441))
        random.shuffle(indexes)
        classify_image_files(category, date, user, indexes[:400], training_folder)
        classify_image_files(category, date, user, indexes[400:440], test_folder)
        
elapse = datetime.now() - begin
print(elapse)

0:00:23.817126


In [1]:
#load to numpy array
from PIL import Image
import numpy as np
from datetime import datetime
import os
from os import listdir
from os.path import isdir, join

begin = datetime.now()
processed_folder = r'C:\Zdata\DL pictures\Processed'
training_folder = join(processed_folder, 'training')
test_folder = join(processed_folder, 'test')

def load_data(data_folder):
    image_list = []
    label_list = []
    with os.scandir(data_folder) as image_files:
        for image_file in image_files:                      
            if not image_file.name.endswith('.jpg'):
                continue
            image_array = np.ndarray((168, 224, 1))
            image = Image.open(join(data_folder, image_file.name))
            width, height = image.size
            for y in range(height):
                for x in range(width):
                    image_array[y][x] = [image.getpixel((x,y))]
            image_list.append(image_array)
            category = image_file.name[:1]
            label_list.append([0 if category == 'A' else 1, 0 if category == 'B' else 1])
    images = np.array(image_list, np.int32)
    labels = np.array(label_list, np.int32)
    return images, labels

training_images, training_labels = load_data(training_folder)
print(training_images.shape)
print(training_labels.shape)

training_images = training_images / 255.0
print(training_images[0])

test_images, test_labels = load_data(test_folder)
print(test_images.shape)
print(test_labels.shape)

test_images = test_images / 255.0
print(test_images[0])

elapse = datetime.now() - begin
print(elapse)

(2400, 168, 224, 1)
(2400, 2)
[[[0.01568627]
  [0.01568627]
  [0.01568627]
  ...
  [0.76862745]
  [0.76862745]
  [0.76862745]]

 [[0.01568627]
  [0.01568627]
  [0.01568627]
  ...
  [0.76862745]
  [0.76862745]
  [0.76862745]]

 [[0.01568627]
  [0.01568627]
  [0.01568627]
  ...
  [0.76862745]
  [0.76862745]
  [0.76862745]]

 ...

 [[0.39215686]
  [0.37647059]
  [0.37254902]
  ...
  [0.38431373]
  [0.38431373]
  [0.38823529]]

 [[0.36862745]
  [0.35294118]
  [0.37647059]
  ...
  [0.37647059]
  [0.37254902]
  [0.37254902]]

 [[0.38431373]
  [0.36862745]
  [0.38431373]
  ...
  [0.36862745]
  [0.36078431]
  [0.35686275]]]
(240, 168, 224, 1)
(240, 2)
[[[0.25098039]
  [0.2627451 ]
  [0.27843137]
  ...
  [0.70980392]
  [0.71372549]
  [0.71764706]]

 [[0.25490196]
  [0.26666667]
  [0.27843137]
  ...
  [0.70588235]
  [0.71372549]
  [0.71764706]]

 [[0.25490196]
  [0.26666667]
  [0.27843137]
  ...
  [0.70588235]
  [0.70980392]
  [0.71372549]]

 ...

 [[0.08235294]
  [0.06666667]
  [0.04313725]
  .

In [6]:
#model
import tensorflow as tf
from tensorflow import keras
from datetime import datetime

begin = datetime.now()
print(tf.__version__)

model = keras.Sequential([
    keras.layers.Conv2D(input_shape=(168, 224, 1), filters=96, kernel_size=11, strides=4, padding="valid", activation=tf.nn.relu),
    keras.layers.AveragePooling2D(pool_size=3, strides=2, padding="valid"),
    keras.layers.Conv2D(filters=256, kernel_size=5, strides=1, padding="same", activation=tf.nn.relu),
    keras.layers.AveragePooling2D(pool_size=3, strides=2, padding="valid"),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

length = len(training_labels)
c = np.c_[training_images.reshape(length, -1), training_labels.reshape(length, -1)]
np.random.shuffle(c)
images = c[:, :37632].reshape((length, 168, 224, 1))
labels = c[:, 37632:].reshape((length, 2))

model.fit(x=images, y=labels, epochs=11, validation_split=0.1)

elapse = datetime.now() - begin
print(elapse)

1.11.0
Train on 2160 samples, validate on 240 samples
Epoch 1/11
2160/2160 [==============================] - 292s 135ms/step - loss: 0.7326 - acc: 0.6856 - val_loss: 0.4776 - val_acc: 0.8250
Epoch 2/11
2160/2160 [==============================] - 271s 125ms/step - loss: 0.4738 - acc: 0.8023 - val_loss: 0.4425 - val_acc: 0.8250
Epoch 3/11
2160/2160 [==============================] - 266s 123ms/step - loss: 0.3850 - acc: 0.8444 - val_loss: 0.3386 - val_acc: 0.8583
Epoch 4/11
2160/2160 [==============================] - 259s 120ms/step - loss: 0.3435 - acc: 0.8704 - val_loss: 0.3075 - val_acc: 0.8792
Epoch 5/11
2160/2160 [==============================] - 262s 121ms/step - loss: 0.2690 - acc: 0.8894 - val_loss: 0.2130 - val_acc: 0.9208
Epoch 6/11
2160/2160 [==============================] - 267s 124ms/step - loss: 0.2188 - acc: 0.9181 - val_loss: 0.1608 - val_acc: 0.9292
Epoch 7/11
2160/2160 [==============================] - 267s 124ms/step - loss: 0.1897 - acc: 0.9306 - val_loss: 0.297

In [7]:
#test
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

240/240 [==============================] - 9s 36ms/step
Test loss: 0.13169637136161327
Test accuracy: 0.9625


In [8]:
#check which images are not predicted correctly
predictions = model.predict(test_images)

file_names = []
with os.scandir(r'C:\Zdata\DL pictures\Processed\test') as image_files:
    for image_file in image_files:                      
        if not image_file.name.endswith('.jpg'):
            continue
        file_names.append(image_file.name)
        
for index in range(0, len(test_labels)):
    result = (test_labels[index][0] - test_labels[index][1]) * (predictions[index][0] - predictions[index][1])
    if (result > 0):
        continue
    print(file_names[index], test_labels[index], predictions[index])

A_1016_0005.jpg [0 1] [0.70872504 0.2912749 ]
A_1016_0023.jpg [0 1] [0.6395397  0.36046025]
A_1017_0050.jpg [0 1] [0.95259416 0.04740584]
A_1018_0365.jpg [0 1] [0.9494691  0.05053084]
B_1016_0311.jpg [1 0] [0.05471027 0.94528973]
B_1017_0376.jpg [1 0] [0.40888053 0.59111947]
B_1018_0062.jpg [1 0] [0.29948953 0.7005105 ]
B_1018_0345.jpg [1 0] [1.3450760e-04 9.9986553e-01]
B_1018_0375.jpg [1 0] [0.4223726  0.57762736]
